# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [ ]:
import pandas as pd

# Reading the tab-delimited files
avocadoDF = pd.read_csv("data/avocado.csv", sep = '\t')
oliveOilDF = pd.read_csv("data/olive_oil.csv", sep = '\t', low_memory=False)
sourdoughDF = pd.read_csv("data/sourdough.csv", sep = '\t')

# Avocados

In [ ]:
# Open the txt file in read mode
with open('data/relevant_avocado_categories.txt', 'r') as file:
    # Read the entire file
    content_avocado = file.read().splitlines()
    
# Print the content
print(content_avocado)


In [ ]:
avocadoDF.info()

In [ ]:
# Subsetting the DataFrame with desired columns

relevant_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']

avocadoDF = avocadoDF[relevant_columns]
avocadoDF.info()

In [ ]:
# Viewing the values in the 'categories_tags' column

avocadoDF['categories_tags'].unique()

In [ ]:
# Dropping the null rows on the 'categories_tags' column

avocadoDF.dropna(subset='categories_tags', inplace=True)
avocadoDF.info()

In [ ]:
# Turning a column of comma-separated tags into a column of lists
avocadoDF['categories_tags'] = avocadoDF['categories_tags'].str.split(',')

In [ ]:
avocadoDF[['categories_tags']]

In [ ]:
categories_list = avocadoDF['categories_tags'].apply(tuple).unique()
display(categories_list, content_avocado)

In [ ]:
# Filtering the DataFrame based on a column of lists

avocadoDF = avocadoDF[avocadoDF['categories_tags'].apply(lambda x: any([i for i in x if i in content_avocado]))]

avocadoDF.info()

In [ ]:
avocadoDF.head()

In [ ]:
# Where do most UK Avocados come from?

top_avocado_origin = avocadoDF[(avocadoDF['countries'] == 'United Kingdom')]['origins_tags'].value_counts().reset_index().iloc[0, 0].split(":")[1]
print(top_avocado_origin)

## Function: get_origin_data

In [ ]:
def get_origin_data(csv_file, txt_file):
    # Read in the tab-delimited data
    df = pd.read_csv(csv_file, sep='\t')
    
    # Subset the large DataFrame
    relevant_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
    df = df[relevant_columns]
    
    # Dropping rows with null values in a particular column
    df.dropna(subset=['categories_tags'], inplace=True)
    
    # Turning a column of comma-separated tags into a column of lists
    df['categories_tags'] = df['categories_tags'].str.split(',')
    
    # Reading data from TXT files for relevant category tags
    with open(txt_file, 'r') as file:
        relevant_categories = file.read().splitlines()
    
    # Filtering a DataFrame based on a column of lists
    df = df[df['categories_tags'].apply(lambda x: any(i in relevant_categories for i in x))]
    
    # Get the country with the highest count
    top_origin = df[df['countries'] == 'United Kingdom']['origins_tags'].value_counts().reset_index().iloc[0, 0].lstrip("en:")
    return top_origin.replace('-', ' ')

# Olive Oil

In [ ]:
# Top Country of Origin for Olive oil

top_olive_oil_origin = get_origin_data('data/olive_oil.csv','data/relevant_olive_oil_categories.txt')
print(top_olive_oil_origin)

# Sourdough

In [ ]:
# Top Country of Origin for Sourdough

top_sourdough_origin = get_origin_data('data/sourdough.csv','data/relevant_sourdough_categories.txt')
print(top_sourdough_origin)